# Core

In [ ]:
#|default_exp core

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
#|export
import os
from collections import Counter
from pathlib import Path
from dotenv import load_dotenv
import logging
logger = logging.getLogger(__name__)

from itertools import count,product
from joblib import Memory
import stringdale
from typing import Dict, List, Iterator, Any

In [ ]:
import pytest

## Git paths and env vars

In [ ]:
#|export

def get_git_root():
        return Path(stringdale.__file__).parent.parent


In [ ]:
assert get_git_root() == Path.cwd().parent

In [ ]:
#|export
def load_env(path=None):
    if path is None:
        path = get_git_root() / '.env.dev'
    else:
        path = Path(path)
    return load_dotenv(path)

In [ ]:
# load_env()
# assert os.getenv('OPENAI_API_KEY') is not None

## Caching

In [ ]:
#|export
import os

In [ ]:
os.environ.get('DISKCACHE','.cache')


'.cache'

In [ ]:
#| export
# Create a cache directory in the user's home directory
cache_location = os.environ.get('DISKCACHE','.cache')
cache_dir = get_git_root() / cache_location
cache_dir.mkdir(parents=True, exist_ok=True)
# Initialize the Memory object for caching
disk_cache = Memory(cache_dir, verbose=0)

## Collection utils

In [ ]:
#| export
def _count_multiplicity(sets):
    """Count how many times each unique item appears across multiple sets.
    """
    # Combine all sets and count frequencies
    counter = Counter()
    for s in sets:
        counter.update(s)
    return counter

def _duplicates(sets):
    """Check if any item appears in more than one set.
    """
    counts = _count_multiplicity(sets)
    return [item for item,count in counts.items() if count > 1]

def merge_list_dicts(dict1, dict2):
    """Merge two dictionaries with list values by concatenating their lists.
    
    Args:
        dict1: defaultdict(list) or dict with list values
        dict2: dict with list values
    
    Returns:
        defaultdict(list) with concatenated lists
    """
    result = defaultdict(list, dict1)
    for k, v in dict2.items():
        result[k].extend(v)
    return result

In [ ]:
from collections import defaultdict

In [ ]:
# Initial dicts
d1 = defaultdict(list, {'a': [1, 2], 'b': [3]})
d2 = {'a': [4], 'c': [5, 6]}

# Merge them
merged = merge_list_dicts(d1, d2)
assert merged == defaultdict(list, {'a': [1, 2, 4], 'b': [3], 'c': [5, 6]})


In [ ]:
#|export
from itertools import product

In [ ]:
#| export
def new_combinations(
    new_params: Dict[str, List],
    old_params: Dict[str, List]
) -> Iterator[Dict[str,Any]]:
    """
    Generate cartesian product of parameters by first choosing new/old for each parameter.
    Skip combinations that only use old parameters.
    
    Args:
        new_params: Dictionary of new parameter values to try
        old_params: Dictionary of previously tried parameter values
    """
    # Get all parameter names
    param_names = list(set(new_params) | set(old_params))
    
    # Generate all possible new/old choices for each parameter (True=new, False=old)
    for choices in product([True, False], repeat=len(param_names)):
        # Skip if all choices are old
        if not any(choices):
            continue
        
        # For each parameter, use either new or old values based on choice
        param_values = []
        for param_name, use_new in zip(param_names, choices):
            if use_new:
                values = new_params.get(param_name, [])
            else:
                values = old_params.get(param_name, [])
            param_values.append(values)
        
        # Generate combinations for this new/old choice pattern
        yield dict(zip(param_names, param_values))


In [ ]:
import json

In [ ]:
# Test 1: Basic functionality
new = {'a': [1, 2], 'b': ['x']}
old = {'a': [2, 3], 'b': ['x', 'y']}

results = list(new_combinations(new, old))
serialized_results = { json.dumps(result,sort_keys=True) for result in results }
assert serialized_results == {'{"a": [1, 2], "b": ["x", "y"]}',
 '{"a": [1, 2], "b": ["x"]}',
 '{"a": [2, 3], "b": ["x"]}'} , results


In [ ]:
# # Should include combinations with at least one new value
# assert {'a': 1, 'b': 'x'} in results
# assert {'a': 1, 'b': 'y'} in results
# assert {'a': 2, 'b': 'x'} in results  # 2 is in both new and old

# # Should not include combinations with only old values
# assert {'a': 3, 'b': 'y'} not in results

# # Test 2: Empty new params
# new = {}
# old = {'a': [1, 2], 'b': ['x']}
# assert list(product_with_memory(new, old)) == []

# # Test 3: Empty old params
# new = {'a': [1, 2], 'b': ['x']}
# old = {}
# results = list(product_with_memory(new, old))
# assert len(results) == 2  # All combinations of new params

# # Test 4: Disjoint parameters
# new = {'a': [1, 2]}
# old = {'b': ['x', 'y']}
# results = list(product_with_memory(new, old))
# assert len(results) == 4  # 2 new 'a' values × 2 old 'b' values
# assert {'a': 1, 'b': 'x'} in results
# assert {'a': 1, 'b': 'y'} in results
# assert {'a': 2, 'b': 'x'} in results
# assert {'a': 2, 'b': 'y'} in results

# # Test 5: Overlapping values
# new = {'a': [1, 2, 3]}
# old = {'a': [3, 4, 5]}
# results = list(product_with_memory(new, old))
# assert {'a': 1} in results
# assert {'a': 2} in results
# assert {'a': 3} in results  # 3 is in both new and old
# assert {'a': 4} not in results
# assert {'a': 5} not in results



In [ ]:
#| export
def dict_cartesian_product(input_dict, keys):
    """
    Generate cartesian products of dictionary values for specified keys while preserving other keys.
    
    Args:
        input_dict (dict): Input dictionary with list values
        keys (list): List of keys to generate products from
        
    Returns:
        list[dict]: List of dictionaries containing all possible combinations
    """
    # Get the lists of values for the specified keys
    value_lists = [input_dict[key] for key in keys]
    
    # Generate cartesian product of the values
    products = product(*value_lists)
    
    # Create dictionaries for each product combination
    result = []
    for values in products:
        # Start with a copy of the original dict
        new_dict = input_dict.copy()
        # Update only the specified keys with their new values
        for key, value in zip(keys, values):
            new_dict[key] = value
        result.append(new_dict)
    
    return result


In [ ]:

# Example usage:
data = {
    'color': ['red', 'blue'],
    'size': ['S', 'M'],
    'style': ['casual', 'formal']
}

keys_to_combine = ['color', 'size']
result = dict_cartesian_product(data, keys_to_combine)
assert result == [{'color': 'red', 'size': 'S', 'style': ['casual', 'formal']},
 {'color': 'red', 'size': 'M', 'style': ['casual', 'formal']},
 {'color': 'blue', 'size': 'S', 'style': ['casual', 'formal']},
 {'color': 'blue', 'size': 'M', 'style': ['casual', 'formal']}],result

In [ ]:
d = defaultdict(list)
d['x'] = [0, 1, 2]
dict_cartesian_product(d,'x')

[defaultdict(list, {'x': 0}),
 defaultdict(list, {'x': 1}),
 defaultdict(list, {'x': 2})]

## Named Lambdas

In [ ]:
#| export
class NamedLambda():
    def __init__(self,name,func):
        self.name = name
        self.func = func

    def __call__(self,*args,**kwargs):
        return self.func(*args,**kwargs)

    def __repr__(self):
        return f'{self.name}'

## Async

### Maybe await

In [ ]:
#| export
from typing import Union,Awaitable,Callable,Any
import inspect
import asyncio

In [ ]:
#| export
async def maybe_await(func_or_coro: Any, args, kwargs) -> Any:
    """
    Prefer __acall__ if it exists, otherwise try normal async/sync calling patterns
    """
    if hasattr(func_or_coro, '__call__'):
        if inspect.iscoroutinefunction(func_or_coro.__call__):
            return await func_or_coro.__call__(*args, **kwargs)
        else:
            return func_or_coro.__call__(*args, **kwargs)
    elif inspect.iscoroutinefunction(func_or_coro):
        return await func_or_coro(*args, **kwargs)
    elif inspect.iscoroutine(func_or_coro):
        return await func_or_coro
    elif callable(func_or_coro):
        return func_or_coro(*args, **kwargs)
    else:
        return func_or_coro

In [ ]:
#| export
async def maybe_await(func_or_coro: Any, args, kwargs) -> Any:
    """
    Prefer __acall__ if it exists, otherwise try normal async/sync calling patterns
    """
    if inspect.iscoroutinefunction(func_or_coro):
        # print('iscoroutinefunction')
        coro = func_or_coro(*args, **kwargs)  # Get the coroutine object
        res = await coro  # Await it before returning
        return res
    elif hasattr(func_or_coro, '__call__'):
        if inspect.iscoroutinefunction(func_or_coro.__call__):
            return await func_or_coro.__call__(*args, **kwargs)
        else:
            return func_or_coro.__call__(*args, **kwargs)
    elif inspect.iscoroutine(func_or_coro):
        return await func_or_coro  # Already a coroutine, just await it
    elif callable(func_or_coro):
        return func_or_coro(*args, **kwargs)
    else:
        return func_or_coro

In [ ]:
class F():
    def __call__(self,x):
        print('acall')
        return x+1

class AsyncF():
    async def __call__(self,x):
        print('call')
        return x+1

async def plus1async(x,**kwargs):
    await asyncio.sleep(0.1)
    return x+1




In [ ]:
assert inspect.iscoroutinefunction(plus1async)
co = await maybe_await(plus1async,[1],{})
co

2

In [ ]:

print(await maybe_await(F(),[1],{}))
print(await maybe_await(AsyncF(),[1],{}))


acall
2
call
2


In [ ]:
co


2

### await_all

In [ ]:
#| export
def get_coro_kwargs(co):
    raw_kwargs = co.cr_frame.f_locals
    explicit_kwargs = raw_kwargs.pop('kwargs',{})
    return {**explicit_kwargs,**raw_kwargs}

async def await_all(coros:List[Awaitable],error_prefix:Union[str,List[str]]=''):
    """
    await all coroutines and raise an exception if any of them raise an exception

    Args:
        coros: list of coroutines to await
        error_prefix: one of:
            - a string that will be formatted with the coroutine's kwargs
            - a list of strings that will be used as the error prefix for each coroutine

    Returns:
        list of results from the coroutines

    Raises:
        Exception: if any of the coroutines raise an exception
    """

    if isinstance(error_prefix,str):
        kwargs = [get_coro_kwargs(co) for co in coros]
        errors = [error_prefix.format(**kw) for kw in kwargs]
    else:
        if not len(error_prefix) == len(coros):
            raise ValueError(f'error_prefix must be a list of the same length as coros, got {len(error_prefix)} and {len(coros)}')
        errors = error_prefix
    
    results = await asyncio.gather(*coros,return_exceptions=True)
    for result,error_prefix in zip(results,errors):
        if isinstance(result,Exception):
            result.args = (f"{error_prefix}\n{result.args[0]}", )+ result.args[1:]
            raise result
    return results



In [ ]:
co = plus1async(1,y=2)
kwargs = get_coro_kwargs(co)
await co
kwargs

{'y': 2, 'x': 1}

In [ ]:
async def bad_coro(x,y=None):
    if x==50:
        raise Exception('bad')
    else:
        return x+y


await await_all([
    bad_coro(10,y=2),
    bad_coro(20,y=2),
])

with pytest.raises(Exception,match='in x=20 y=None'):
    await await_all([
        bad_coro(10,y=2),
        bad_coro(20,y=None),
    ],'in x={x} y={y}')


with pytest.raises(Exception,match='in x=50 y=10\nbad'):
    await await_all([
        bad_coro(10,y=2),
        bad_coro(50,y=10),
    ],'in x={x} y={y}')


### Semaphore

In [ ]:
#| export
from contextlib import asynccontextmanager
from typing import Optional
import asyncio
from functools import wraps
from typing import TypeVar, Callable, Any, Union, Type
import time
from contextlib import contextmanager

In [ ]:
#| export
@contextmanager
def timeit(assert_duration:float=None,tolerance:float=0.05):
    start_time = time.time()
    yield
    end_time = time.time()
    duration = end_time - start_time
    if assert_duration is not None:
        assert assert_duration - tolerance < duration < assert_duration + tolerance, f"Expected duration {assert_duration:.2f}s but got {duration:.2f}s"
    print(f"Block executed in {duration:.2f} seconds")

In [ ]:
#| export
class OptionalSemaphore():
    def __init__(self, concurrency:int=0):
        self.set_concurrency(concurrency)
    
    def set_concurrency(self, concurrency:int):
        if concurrency > 0:
            self.semaphore = asyncio.Semaphore(concurrency)
        else:
            self.semaphore = None

    async def __aenter__(self):
        if self.semaphore is not None:
            await self.semaphore.acquire()

    async def __aexit__(self, exc_type, exc_value, traceback):
        if self.semaphore is not None:
            self.semaphore.release()

In [ ]:
sem = OptionalSemaphore(2)

async def sleeper():
    async with sem:
        await asyncio.sleep(0.2)


async def sleep3():

    await asyncio.gather(
        sleeper(),
        sleeper(),
        sleeper(),
    )

with timeit(assert_duration=0.4):
    await sleep3()

Block executed in 0.40 seconds


In [ ]:
#| export
def method_with_semaphore(concurrency:int=0):
    def wrapper(func):
        if isinstance(concurrency, OptionalSemaphore):
            sem = concurrency
        else:
            sem = OptionalSemaphore(concurrency)
            
        # Add the semaphore as an attribute to the wrapped function
        func._semaphore = sem
            
        @wraps(func)
        async def wrapped_func(*args, **kwargs):
            # Check for concurrency override on the function
            if hasattr(func, 'concurrency_limit'):
                sem.set_concurrency(func.concurrency_limit)
            # For methods, also check the class instance
            elif args and hasattr(args[0], 'concurrency_limit'):
                sem.set_concurrency(args[0].concurrency_limit)
                
            async with sem:
                return await func(*args, **kwargs)
        
        # Expose the semaphore on the wrapped function too
        wrapped_func._semaphore = sem
        return wrapped_func
    return wrapper

In [ ]:
@method_with_semaphore(2)
async def sleeper():
    await asyncio.sleep(0.2)

with timeit(assert_duration=0.4):
    await asyncio.gather(
        sleeper(),
        sleeper(),
        sleeper(),
    )


shared_sem = OptionalSemaphore(1)

@method_with_semaphore(shared_sem)
async def sleeper1():
    await asyncio.sleep(0.2)

@method_with_semaphore(shared_sem)
async def sleeper2():
    await asyncio.sleep(0.3)
    

with timeit(assert_duration=0.5):
    await asyncio.gather(
        sleeper1(),
        sleeper2(),
    )



Block executed in 0.40 seconds
Block executed in 0.50 seconds


In [ ]:
#| export

def _is_function(obj):
    return callable(obj) and not isinstance(obj,type)

def _is_class(obj):
    return isinstance(obj,type)

def _is_method(obj,has_method:str=None):
    return _is_class(obj) and _is_function(getattr(obj,has_method))


def semaphore_decorator(concurrency:int=0, method_name:str=None):
    """A decorator that limits concurrent execution of functions or class methods using an async semaphore.
    
    This decorator can be applied to both async functions and classes. When applied to a class,
    it can limit concurrent executions of a specific method (defaults to __call__) across all
    instances of that class.
    
    Parameters
    ----------
    concurrency : int, default=0
        The maximum number of concurrent executions allowed. If 0, no concurrency limit is applied.
        
    method_name : str, optional
        When decorating a class, specifies which method to apply the semaphore to.
        Defaults to '__call__'. Must be None when decorating a function.
    """
    def wrapper(obj):
        nonlocal method_name
        if _is_function(obj):
            if not method_name is None:
                raise ValueError(f"method_name must be None for functions, got {method_name}")
            return method_with_semaphore(concurrency)(obj)
        elif _is_class(obj):
            method_name = method_name or '__call__'
            if not _is_method(obj, method_name):
                raise ValueError(f"method_name must be a method of {obj.__name__}, got {method_name}")

            init = obj.__init__ 
            shared_instance_semaphore = OptionalSemaphore(concurrency)
            method = getattr(obj, method_name)
            
            @wraps(init)
            def new_init(*args, **kwargs):
                init(*args, **kwargs)
                semaphored_method = method_with_semaphore(shared_instance_semaphore)(method)
                setattr(obj, method_name, semaphored_method)
            
            obj.__init__ = new_init
            return obj
        else:
            raise ValueError(f"obj must be a function or class, got {type(obj)}")
    return wrapper

In [ ]:
@semaphore_decorator(2)
async def sleeper():
    await asyncio.sleep(0.2)

with timeit(assert_duration=0.4):
    await asyncio.gather(
        sleeper(),
        sleeper(),
        sleeper(),
    )



Block executed in 0.40 seconds


In [ ]:
sleeper._semaphore.set_concurrency(1)

with timeit(assert_duration=0.6):
    await asyncio.gather(
        sleeper(),
        sleeper(),
        sleeper(),
    )


Block executed in 0.60 seconds


In [ ]:
@semaphore_decorator(1,'shmall')
@semaphore_decorator(2)
class A():
    async def __call__(self):
        await asyncio.sleep(0.2)

    async def shmall(self):
        await asyncio.sleep(0.1)

a1 = A()
a2 = A()
a3 = A()

with timeit(assert_duration=0.3):
    await asyncio.gather(
        a1.shmall(),
        a2.shmall(),
        a3.shmall(),
    )


with timeit(assert_duration=0.4):
    await asyncio.gather(
        a1(),
        a2(),
        a3(),
    )



Block executed in 0.30 seconds
Block executed in 0.40 seconds


In [ ]:
A.concurrency_limit = 3
with timeit(assert_duration=0.1):
    await asyncio.gather(
        a1.shmall(),
        a2.shmall(),
        a3.shmall(),
    )


with timeit(assert_duration=0.2):
    await asyncio.gather(
        a1(),
        a2(),
        a3(),
    )

# TODO in future, allow resetting different concurrency limits on different methods

Block executed in 0.10 seconds
Block executed in 0.20 seconds


## Mocking functions

In [ ]:
#|export
import pickle
from typing import Dict, Any, Tuple
import inspect
from joblib.memory import MemorizedFunc
import json
from pydantic import BaseModel

In [ ]:
#|export
def get_input_output_from_cache(func) -> Dict[Tuple, Any]:
    """Extracts input/output pairs from a joblib Memory cache
    
    Args:
        func: The cached function (must be decorated with joblib.Memory.cache)
        
    Returns:
        Dict[Tuple, Any]: Dictionary mapping input tuples (args, kwargs) to outputs
    """
    # Check if function is a MemorizedFunc type
    if not isinstance(func, MemorizedFunc):
        raise ValueError(f"Function {func.__name__} doesn't appear to be cached with joblib.Memory")
    
    # Get all cache directories for this function
    backend_path = Path(func.store_backend.location)
    func_name = func.func.__name__
    func_cache_dirs = list(backend_path.glob(f"*/{func_name}"))
    
    if not func_cache_dirs:
        raise ValueError(f"No cache found for function {func_name} in {func.store_backend.location}")
    
    cache_dict = {}
    for func_cache_dir in func_cache_dirs:
        for output_file in func_cache_dir.glob('**/output.pkl'):
            # Get output from output.pkl
            with open(output_file, 'rb') as f:
                output = pickle.load(f)
            
            # Get inputs from metadata.json
            metadata_file = output_file.parent / 'metadata.json'
            if metadata_file.exists():
                metadata = json.loads(metadata_file.read_text())
                input_args = metadata.get('input_args', {})
                # Create a hashable key from the input args
                cache_key = frozenset(input_args.items())
                cache_dict[cache_key] = output
        
    return cache_dict

In [ ]:
#|export
def mock_from_dict(func, cache_dict: Dict[Tuple, Any], call_on_missing: bool = True):
    """Creates a mock function that uses cached results from a dictionary,
    optionally calling the original function for missing inputs
    
    Args:
        func: The original function to mock
        cache_dict: Dictionary mapping input tuples (args, kwargs) to outputs
        call_on_missing: If True, calls original function when input not in cache
        
    Returns:
        callable: A function that returns cached results or calls the original
    """
    # Get the original function's signature
    sig = inspect.signature(func.func if isinstance(func, MemorizedFunc) else func)
    
    def mocked_func(*args, **kwargs):
        # Convert args to kwargs using the function signature
        bound_args = sig.bind(*args, **kwargs)
        all_kwargs = bound_args.arguments
        
        # Create cache key from kwargs
        cache_key = frozenset(all_kwargs.items())
        
        if cache_key in cache_dict:
            return cache_dict[cache_key]
        elif call_on_missing:
            return func(**all_kwargs)
        else:
            raise KeyError(f"No cached result for inputs: {all_kwargs}")
    
    return mocked_func

In [ ]:
# Example usage:
from joblib import Memory
from unittest.mock import patch


In [ ]:

# Set up the cache
memory = Memory(Path.home() / '.cache', verbose=0)
# memory.clear()
# Define and cache the function
@memory.cache
def expensive_api_call(x, y):
    # Simulate expensive API call
    print(f'calling expensive_api_call({x}, {y})')  # To show when real function is called
    return x + y

expensive_api_call(1, 2)
expensive_api_call(3, 4)

cache_dict = get_input_output_from_cache(expensive_api_call)
print(cache_dict)

# Use the mock in a context manager
mock_func = mock_from_dict(expensive_api_call, cache_dict)

with patch('__main__.expensive_api_call',mock_func) as mock_api:
    # Configure mock to use our cached results
    
    # These calls will use cached results without hitting the real API
    assert mock_func(1, 2) == 3
    assert mock_func(3, 4) == 7
    

{frozenset({('x', '1'), ('y', '2')}): 3, frozenset({('x', '3'), ('y', '4')}): 7}


## Logging

In [ ]:
#| export
from contextlib import contextmanager
import logging

# Create a handler that flushes after each write
class FlushingStreamHandler(logging.StreamHandler):
    def emit(self, record):
        super().emit(record)
        self.flush()

@contextmanager
def checkLogs(level: int=logging.DEBUG, name :str='__main__', toFile: str|Path=None,format="%(message)s"):
    """context manager for temporarily changing logging levels. used for debugging purposes

    Args:
        level (logging.Level: optional): logging level to change the logger to. Defaults to logging.DEBUG.
        name (str: optional): module name to raise logging level for. Defaults to root logger
        toFile (Path: optional): File to output logs to. Defaults to None
        

    Yields:
        [logging.Logger]: the logger object that we raised the level of
    """
    logger = logging.getLogger(name)
    current_level = logger.getEffectiveLevel()
    format = "%(name)s - %(levelname)s - %(message)s"
    logger.setLevel(level)
    if len(logger.handlers) == 0:
        pass
        sh = FlushingStreamHandler()
        sh.setFormatter(logging.Formatter(format))
        logger.addHandler(sh)
    if toFile != None:
        fh = logging.FileHandler(toFile)
        fh.setFormatter(logging.Formatter(format))
        logger.addHandler(fh)
    try:
        yield logger
    finally:
        logger.setLevel(current_level)
        if toFile != None:
            logger.removeHandler(fh)
        if len(logger.handlers) == 1:
            logger.handlers= []

## Jinja2

In [ ]:
#| export
from jinja2 import Template, Environment, PackageLoader, meta

def jinja_undeclared_vars(template):
    """Computes all undeclared vars in a jinja template

    Args:
        template (Path or str): Path to file of template or string with the template content

    Returns:
        set: set of all undeclared vars
    """
    if isinstance(template, Path):
        template = template.read_text()
    env = Environment()
    parsed_content = env.parse(template)
    return meta.find_undeclared_variables(parsed_content)


In [ ]:
#| export
def jinja_render(template, params: dict, silent=True, to_file: Path = None):
    """renders a jinja template

    Args:
        template (Path or str): Path to file of template or string with the template content
        params (Dict): parameter dictionary with the variables to render into the template
        silent (Bool, Optional): Whether to print the rendered template to screen, defaults to False
        to_file (Path, Optional): If a path is supplied, prints the template to the file of said path

    Returns:
        set: set of all undeclared vars
    """
    if isinstance(template, Path):
        template = template.read_text()
    instance_str = Template(template).render(**params)

    if not silent:
        print(instance_str)

    if to_file:
        to_file.write_text(instance_str)
        return None
    else:
        return instance_str

In [ ]:
#| export
from typing import Any,Dict
from copy import deepcopy
from textwrap import dedent


In [ ]:
#| export

def _clean_whitespace(content: str) -> str:
    """Clean whitespace in content by removing empty lines at start/end and dedenting.
    
    Args:
        content: String content to clean
        
    Returns:
        Cleaned content string
    """
    if not isinstance(content, str):
        return content
    return dedent(content).strip()

def json_undeclared_vars(data: Any) -> set:
    """Recursively traverse a JSON-like object and collect all undeclared Jinja variables.
    
    Args:
        data: JSON-like object to analyze
        
    Returns:
        set: Set of all undeclared variables found in the structure
        
    Examples:
        >>> data = {
        ...     "user_{{name}}": {
        ...         "age": "{{age}} years old",
        ...         "greeting": "Hello {{name}}!"
        ...     }
        ... }
        >>> json_undeclared_vars(data)
        {'name', 'age'}
    """
    undeclared = set()
    
    if isinstance(data, dict):
        # Check keys and values
        for k, v in data.items():
            if isinstance(k, str):
                undeclared.update(jinja_undeclared_vars(k))
            undeclared.update(json_undeclared_vars(v))
    elif isinstance(data, list):
        # Check list elements
        for item in data:
            undeclared.update(json_undeclared_vars(item))
    elif isinstance(data, str):
        # Check string for variables
        undeclared.update(jinja_undeclared_vars(data))
    
    return undeclared

def json_render(data: Any, context: Dict[str, Any],clean_whitespace: bool = True) -> Any:
    """Recursively traverse a JSON-like object and render all strings using Jinja.
    
    Args:
        data: JSON-like object to render
        context: Dictionary of variables to use for rendering
        
    Returns:
        Rendered copy of the input data structure
        
    Examples:
        >>> context = {"name": "Alice", "age": 30}
        >>> data = {
        ...     "user_{{name}}": {
        ...         "age": "{{age}} years old",
        ...         "greeting": "Hello {{name}}!"
        ...     }
        ... }
        >>> json_render(data, context)
        {
            "user_Alice": {
                "age": "30 years old",
                "greeting": "Hello Alice!"
            }
        }
    """
    # Deep copy to avoid modifying original
    data = deepcopy(data)
    
    if isinstance(data, dict):
        # Create new dict with rendered keys and recursively rendered values
        return {
            jinja_render(str(k), context): json_render(v, context,clean_whitespace) 
            for k, v in data.items()
        }
    elif isinstance(data, list):
        # Recursively render list elements
        return [json_render(item, context,clean_whitespace) for item in data]
    elif isinstance(data, str):
        # Render string values
        if clean_whitespace:
            return _clean_whitespace(jinja_render(data, context))
        else:
            return jinja_render(data, context)
    else:
        # Return non-string values unchanged
        return data

In [ ]:
data = {
        "{{user_name}}": {
                 "age": ["{{age}} years old"],
                 "greeting": """
                 Hello
                 {{name}}!
                 """,
             }
        }

assert json_undeclared_vars(data) == {'user_name', 'age', 'name'}
context = {"user_name": "John", "age": 30, "name": "Alice"}
rendered = json_render(data, context)
assert rendered == {'John': {'age': ['30 years old'], 'greeting': 'Hello\nAlice!'}},rendered


## Better exception messages

In [ ]:
#| export
from functools import wraps
import inspect
from typing import Callable, TypeVar, ParamSpec
from stringdale.core import jinja_render


In [ ]:
#| export
P = ParamSpec("P")
R = TypeVar("R")

def wrap_exception(template: str, verbose: bool = False) -> Callable[[Callable[P, R]], Callable[P, R]]:
    """Decorator that adds context to exceptions using a template string.
    
    Args:
        template: A jinja template string that can reference function kwargs
        verbose: If True, includes all local variables in error message
    
    Example:
        @helpful_context("Failed to process {{filename}}")
        def process_file(filename: str):
            ...
    """
    def decorator(func: Callable[P, R]) -> Callable[P, R]:
        required_vars = jinja_undeclared_vars(template)
        if 'self' in required_vars:
            raise ValueError("self is not allowed in template strings")
        @wraps(func)
        def wrapper(*args: P.args, **kwargs: P.kwargs) -> R:
            try:
                return func(*args, **kwargs)
            except Exception as e:
                # Get bound arguments
                sig = inspect.signature(func)
                bound_args = sig.bind(*args, **kwargs)
                bound_args.apply_defaults()
                
                # Get local variables at time of error
                frame = inspect.trace()[-2][0]
                local_vars = {k:v for k,v in frame.f_locals.items() 
                            if not k.startswith('__')}
                
                # Combine args and locals for template rendering
                template_vars = {**bound_args.arguments, **local_vars}
                template_vars = {k:repr(v) for k,v in template_vars.items()}

                # cant render self since it clashes with the render class of jinja
                if 'self' in template_vars:
                    template_vars.pop('self')
                                
                # Render template with combined context
                context_msg = jinja_render(template, template_vars)
                
                # Build error message
                error_parts = [
                    f"{context_msg}",
                ]
                
                # Add locals if verbose
                if verbose:
                    error_parts.extend([
                        f"\nLocal variables in {func.__name__}() at time of error:",
                        *[f"  {k} = {repr(v)}" for k,v in local_vars.items()]
                    ])
                
                # Add original error at the end
                error_parts.append(f"{str(e)}")
                
                raise type(e)("\n".join(error_parts)) from e
                
        return wrapper
    return decorator

In [ ]:
'\n'.join([])

''

In [ ]:
@wrap_exception("Failed to read config file {{filepath}}. Local x value was {{x}}", verbose=True)
def read_config(filepath: str, encoding: str = 'utf-8') -> dict:
    x = 42  # example local variable
    data = {'test': 123}  # another local
    with open(filepath, encoding=encoding) as f:
        return json.load(f)

# When called with invalid parameters:
try:
    read_config('missing.json')
except FileNotFoundError as e:
    print(e)
    # Output will be something like:
    # Failed to read config file missing.json. Local x value was 42
    # Original error: [Errno 2] No such file or directory: 'missing.json'
    # 
    # Local variables at time of error:
    #   filepath = 'missing.json'
    #   encoding = 'utf-8'
    #   x = 42
    #   data = {'test': 123}

Failed to read config file 'missing.json'. Local x value was 42

Local variables in read_config() at time of error:
  filepath = 'missing.json'
  encoding = 'utf-8'
  x = 42
  data = {'test': 123}
[Errno 2] No such file or directory: 'missing.json'


## Validate Input

In [ ]:
#| export
def is_valid_object(obj, model):
    try:
        model.model_validate(obj)
        return True
    except Exception as e:
        return False

In [ ]:
class Foo(BaseModel):
    a: int
    b: str


assert is_valid_object({"a": 10, "b": "adf"}, Foo)
assert not is_valid_object({"a": "hehe", "b": "adf"}, Foo)
assert is_valid_object({"a": 10, "b": "adf", "c": 3}, Foo)

## Working with Missing values

In [ ]:
#| export
from enum import Enum
from pydantic import Field
from typing import Union

#| export
def get_missing(model, keys=None):
    """
    Recursively check a model for Missing or None values and return a list of keys with Missing/None values.
    
    Args:
        model: A pydantic model instance or dict
        keys: Optional list of specific keys to check. If None, checks all keys.
    
    Returns:
        list: List of keys (as strings) that have Missing or None values
    """
    missing_keys = []
    
    # Convert BaseModel to dict if needed
    if isinstance(model, BaseModel):
        model = model.model_dump()
    
    # If keys is provided, only check those specific keys
    items_to_check = model.items()
    if keys is not None:
        items_to_check = [(k, model.get(k)) for k in keys if k in model]

    for key, value in items_to_check:
        # Check for Missing enum or None
        if value is None:
            missing_keys.append(key)
        # Recursively check nested structures
        elif isinstance(value, (dict, BaseModel)):
            nested_keys = None if keys is None else [
                k.split('.', 1)[1] for k in (keys or [])
                if k.startswith(f"{key}.") and '.' in k
            ]
            if nested_keys != []:  # Only recurse if we have nested keys to check or keys is None
                nested_missing = get_missing(value, nested_keys)
                missing_keys.extend(f"{key}.{k}" for k in nested_missing)
                
    return missing_keys

def has_missing(model, keys=None):
    return len(get_missing(model, keys)) > 0


In [ ]:
#| export
from sqlmodel import Field,SQLModel
from typing import Optional

In [ ]:

class UserData(SQLModel):
    name: Optional[str] = Field(None, description='The name of the user')
    age: Optional[int] = Field(None, description='The age of the user')
    email: Optional[str] = Field(None, description='The email of the user')



In [ ]:
# Test empty dict
assert has_missing(UserData()) == True

# Test basic dict with Missing value
test_dict = {"name": None, "age": 25}
assert get_missing(test_dict) == ["name"]

# Test dict with no Missing values
test_dict = {"name": "John", "age": 25}
assert get_missing(test_dict) == []

# Test nested dict with Missing values
test_nested = {
    "user": {
        "name": "John",
        "details": {
            "email": None,
            "phone": "123-456-7890"
        }
    }
}
assert get_missing(test_nested) == ["user.details.email"]

# Test with Pydantic model
test_model = UserData(name="John", age=None, email="john@example.com")
assert get_missing(test_model) == ["age"]

# Test with all Missing values
test_all_missing = UserData(name=None, age=None, email=None)
assert sorted(get_missing(test_all_missing)) == sorted(["name", "age", "email"])

test_partial = UserData(name=None, age=25, email=None)
assert get_missing(test_partial, keys=["name", "age"]) == ["name"] 

# Export

In [ ]:
#|hide
import nbdev; nbdev.nbdev_export()
     